# 셀 서식 지정하여 엑셀파일 만들기

In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime

In [ ]:
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook
# from openpyxl.utils import get_column_letter
# from openpyxl.utils.cell import coordinate_from_string

# 데이터프레임 생성
df = pd.DataFrame({
    '키' : ["12345678", "23456789", "34567890"],
    '이름': ['전병만', '호호호', '하하하'],
    '번호': [121157498, 21654648455, 38454864513],
    '학교': ['서울대학교', '한양대학교', '연세대학교'],
    '등급': ['A', 'B', 'C'],
    '국어': [95, 80, 70],
    '영어': [80, 90, 75],
    '수학': [90, 85, 60],
    '과학': [0.1, 20, 18.63],
    '등록일': ['2021-05-01', "", '2021-05-03']
})

# 엑셀 파일 생성
wb = Workbook()
ws1 = wb.active
ws1.title = "무담보"

# 데이터프레임 삽입
for r in dataframe_to_rows(df, index=False, header=True):
    ws1.append(r)


# 스타일 지정
# # 폰트
font = Font(name='NN30', size=10, color='FF000000')
# 폰트 (데이터 부분)
font_data = Font(name='NN30', size=10)
# 배경색
fill = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6')
# 테두리
border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
# 정렬
alignment = Alignment(horizontal='center', vertical='center')

# 컬럼명 스타일 적용
for cell in ws1[1]:
    cell.font = Font(name='NN30', size=10, color='FFFFFF', bold=True)
    cell.fill = fill
    cell.border = border
    cell.alignment = alignment

# 데이터 부분 스타일 적용
for row in ws1.iter_rows(min_row=2):
    for cell in row:
        # 숫자 데이터인 경우
        if isinstance(cell.value, (int, float)):
            cell.font = font_data
            cell.number_format = '#,##0' if isinstance(cell.value, int) else '#,##0.00'
            cell.alignment = Alignment(horizontal='right', vertical='center')
        # 날짜 데이터인 경우
        elif isinstance(cell.value, pd.Timestamp):
            cell.font = font_data
            cell.number_format = 'yyyy-mm-dd'
            cell.alignment = alignment
        # 나머지 문자열 데이터인 경우
        else:
            cell.font = font_data
            cell.alignment = alignment
        cell.border = border
        
# 열 너비 자동 조정(단, 영어와 숫자만 정확히 반영됨 (한글>영어))
for column_cells in ws1.columns:
    if re.search("원금|번호", column_cells[0].value) : # 컬럼명에 따라
        length = max(len(str(cell.value))+2 for cell in column_cells)
        ws1.column_dimensions[column_cells[0].column_letter].width = length
# 파일 저장
wb.save('example.xlsx')


# 파일의 셀 서식을 유지하기 

In [34]:
import pandas as pd
import numpy as np
import pickle
import re, os
from os.path import join
from tqdm import tqdm
from datetime import datetime
from openpyxl import Workbook
from openpyxl import load_workbook # 파일 불러옥
from openpyxl.styles import NamedStyle
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.styles.numbers import NumberFormat
from openpyxl.utils.dataframe import dataframe_to_rows
from copy import copy

In [137]:
df = pd.DataFrame({
    '키' : ["12345678", "23456789", "34567890"],
    '이름': ['전병만', '호호호', '호호호'],
    '번호': [121157498, 21654648455, 164567],
    '학교': ['서울대학교', '한양대학교', '연세대학교'],
    '등급': ['A', 'B', 'C'],
    '국어': [95, 80, 70],
    '영어': [80, 90, 75],
    '수학': [90, 85, 60],
    '과학': [0.1, 20, 18.63],
    '등록일': ['2021-05-01', "", '2021-05-03']
})

In [12]:
wb = Workbook()
ws1 = wb.active
ws1.title = "요약"
ws1.freeze_panes = 'A2'
for r in dataframe_to_rows(df, index=False, header=True) : 
    ws1.append(r)
wb.save("ex.xlsx")


In [35]:
# 서식 있는 엑셀파일을 열어서 데이터행 서식을 그대로 유지하기 
# 엑셀 파일 열기
wb = load_workbook('ex.xlsx', data_only=False)

# 활성 시트 선택
ws = wb.active
ws.append(["987654321", "가갸너", 5486413188482,'김포대학교','D', 88,76,95,1.1,"2022-03-21 13:36:56"])

# 데이터 행의 첫 번째 행의 서식 가져오기
first_row_style = []
for col in range(1, ws.max_column + 1):
    cell = ws.cell(row=2, column=col)

    font = copy(cell.font)
    fill = copy(cell.fill)
    alignment = copy(cell.alignment)
    number_format = copy(cell.number_format)

    style = NamedStyle(name="style_{0}".format(col))
    style.font = font
    style.fill = fill
    style.alignment = alignment
    style.number_format = number_format
    first_row_style.append(style)

# 모든 데이터 행에 첫 번째 행의 서식 적용
for row in range(3, ws.max_row + 1):
    for col in range(1, ws.max_column + 1):
        cell = ws.cell(row=row, column=col)
        cell.style = first_row_style[col - 1]
wb.save("ex1.xlsx")

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   키       3 non-null      object 
 1   이름      3 non-null      object 
 2   번호      3 non-null      int64  
 3   학교      3 non-null      object 
 4   등급      3 non-null      object 
 5   국어      3 non-null      int64  
 6   영어      3 non-null      int64  
 7   수학      3 non-null      int64  
 8   과학      3 non-null      float64
 9   등록일     3 non-null      object 
dtypes: float64(1), int64(4), object(5)
memory usage: 368.0+ bytes
